In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [2]:
# Sample text data for training (replace with your own dataset)
corpus = [
    'This is a sample sentence.',
    'Another example sentence.',
    'Yet another sentence for the model.',
    # Add more sentences as needed
]

In [3]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [4]:
# Create input sequences and labels
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [5]:
max_sequence_length = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

In [6]:
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [7]:
# Build the LSTM model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

In [8]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
# Train the model
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 2.4878 - accuracy: 0.0909
Epoch 2/100
1/1 [==============================] - 0s 16ms/step - loss: 2.4778 - accuracy: 0.1818
Epoch 3/100
1/1 [==============================] - 0s 19ms/step - loss: 2.4679 - accuracy: 0.2727
Epoch 4/100
1/1 [==============================] - 0s 18ms/step - loss: 2.4580 - accuracy: 0.4545
Epoch 5/100
1/1 [==============================] - 0s 17ms/step - loss: 2.4478 - accuracy: 0.4545
Epoch 6/100
1/1 [==============================] - 0s 18ms/step - loss: 2.4374 - accuracy: 0.3636
Epoch 7/100
1/1 [==============================] - 0s 16ms/step - loss: 2.4264 - accuracy: 0.3636
Epoch 8/100
1/1 [==============================] - 0s 17ms/step - loss: 2.4149 - accuracy: 0.4545
Epoch 9/100
1/1 [==============================] - 0s 16ms/step - loss: 2.4025 - accuracy: 0.4545
Epoch 10/100
1/1 [==============================] - 0s 17ms/step - loss: 2.3892 - accuracy: 0.3636
Epoch 11/100
1/1 [===

In [10]:
# Function to generate the next word
def generate_next_word(seed_text, model, tokenizer, max_sequence_length):
    for _ in range(3):  # Adjust the number of words to generate
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [11]:
# Test the model
seed_text = "This is a"
generated_text = generate_next_word(seed_text, model, tokenizer, max_sequence_length)
print(generated_text)

1/1 [==============================] - 0s 32ms/step
This is a sample sentence sentence
